In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In this exercise, you will create a movie recommendation engine from the MovieLens data. You can download that data by clicking here: MovieLens.

#Movie Recommendation Engine

##a. Prepare Data

Load the data from the ratings.csv and movies.csv files and combine them on movieId. The resultant data set should contain all of the user ratings and include movie titles. The schema should look something like this.

In [3]:
spark = SparkSession.builder.appName('Ops').getOrCreate()

In [4]:
df1 = spark.read.csv('/FileStore/tables/650/data/movies/ratings.csv',inferSchema=True,header=True)
df2 = spark.read.csv('/FileStore/tables/650/data/movies/movies.csv',inferSchema=True,header=True)


In [5]:
df = df1.join(df2, 'movieID')

In [6]:
df.printSchema()

root
-- movieId: integer (nullable = true)
-- userId: integer (nullable = true)
-- rating: double (nullable = true)
-- timestamp: integer (nullable = true)
-- title: string (nullable = true)
-- genres: string (nullable = true)

In [7]:
df = df.withColumn("rating", df["rating"].cast('integer'))

## b. Train Recommender

Using the data you prepared in the last step, create a movie recommendation model using collaborative filtering. Spark’s collaborative filtering documentation provides a template for building and testing this model.

Before you train the recommendation model, split the data into a training dataset and a testing dataset using the randomSplit dataframe method. Use 80% of your data for training and 20% for testing.

After fitting your model using the training dataset, calculate the predictions on the test dataset and use the RegressionEvaluator to calculate the root-mean-square error of the model.

As a reminder, Spark’s collaborative filtering documentation will be helpful in completing this task.

In [9]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.ml.evaluation import RegressionEvaluator

In [10]:
training, test = df.randomSplit([0.8, 0.2])

In [11]:
als = ALS()\
.setMaxIter(5)\
.setRegParam(0.01)\
.setUserCol("userId")\
.setItemCol("movieId")\
.setRatingCol("rating")\
.setColdStartStrategy("drop")
alsModel = als.fit(training)
predictions = alsModel.transform(test)

In [12]:
evaluator = RegressionEvaluator()\
.setMetricName("rmse")\
.setLabelCol("rating")\
.setPredictionCol("prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = %f" % rmse)

Root-mean-square error = 1.129730

## c. Generate top 10 movie recommendations

In [14]:
userRecommendations = alsModel.recommendForAllUsers(10)
movieRecommendations = alsModel.recommendForAllItems(10)